In [23]:
import os
import shutil
import random

def partition_images(base_dir, train_dir, test_dir, train_ratio):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    all_images = os.listdir(base_dir)
    random.shuffle(all_images)

    train_count = int(len(all_images) * train_ratio)
    train_images = all_images[:train_count]
    test_images = all_images[train_count:]

    for image in train_images:
        shutil.copyfile(os.path.join(base_dir, image), os.path.join(train_dir, image))

    for image in test_images:
        shutil.copyfile(os.path.join(base_dir, image), os.path.join(test_dir, image))

partition_images('images', 'train', 'test', 0.8)


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
import cv2
import numpy as np
from torchvision import transforms
import random

# ensure determinism
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)\.png"
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image)
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([alfa_x, beta_x]) # returns alfa x and betax as the labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 8
learning_rate = 0.001
num_epochs = 250

model = CNN().to(device)
criterion = nn.L1Loss() # MAE
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

image_dir = "train"
dataset = CustomDataset(image_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

total_steps = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")

        # if (i + 1) % 10 == 0:
        #     print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")

filename = f"cnn_model_bs{batch_size}_lr{learning_rate}_e{num_epochs}.pth"

torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
    }, filename)

torch.save(model.state_dict(), filename)

# checkpoint = torch.load(filename)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# num_epochs = checkpoint['epoch']
# loss = checkpoint['loss']

Epoch [1/25], Step [1/140], Loss: 154.0125
Epoch [1/25], Step [2/140], Loss: 254.8400
Epoch [1/25], Step [3/140], Loss: 119.3179
Epoch [1/25], Step [4/140], Loss: 113.7042
Epoch [1/25], Step [5/140], Loss: 84.7923
Epoch [1/25], Step [6/140], Loss: 137.2838
Epoch [1/25], Step [7/140], Loss: 105.3344
Epoch [1/25], Step [8/140], Loss: 77.9037
Epoch [1/25], Step [9/140], Loss: 78.1877
Epoch [1/25], Step [10/140], Loss: 80.2626
Epoch [1/25], Step [11/140], Loss: 68.7683
Epoch [1/25], Step [12/140], Loss: 51.0361
Epoch [1/25], Step [13/140], Loss: 38.5167
Epoch [1/25], Step [14/140], Loss: 46.8061
Epoch [1/25], Step [15/140], Loss: 38.3636
Epoch [1/25], Step [16/140], Loss: 78.6200
Epoch [1/25], Step [17/140], Loss: 98.8882
Epoch [1/25], Step [18/140], Loss: 50.7807
Epoch [1/25], Step [19/140], Loss: 80.1082
Epoch [1/25], Step [20/140], Loss: 33.5948
Epoch [1/25], Step [21/140], Loss: 45.5939
Epoch [1/25], Step [22/140], Loss: 73.5188
Epoch [1/25], Step [23/140], Loss: 78.4745
Epoch [1/25], 

KeyboardInterrupt: 

In [26]:
import torch
import os
from torch.utils.data import DataLoader
import math
import numpy as np

def evaluate_model(model, test_dir):
    model.eval()
    test_dataset = CustomDataset(test_dir)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    total_mae = torch.zeros(2, device=device)   # Added device argument
    total_mape = torch.zeros(2, device=device)  # Added device argument
    total_smape = torch.zeros(2, device=device) # Added device argument
    total_mse = torch.zeros(2, device=device)   # Added device argument
    total_count = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            absolute_error = torch.abs(outputs - labels)  # Absolute error
            total_mae += absolute_error.sum(dim=0)

            non_zero_mask = torch.abs(labels) > 1e-8  # Only calculate percentage for non-zero labels
            percentage_error = (absolute_error / torch.abs(labels)) * 100  # Percentage error
            total_mape += (percentage_error * non_zero_mask).sum(dim=0)

            smape = 200.0 * torch.abs(outputs - labels) / (torch.abs(outputs) + torch.abs(labels) + torch.finfo(torch.float32).eps)
            total_smape += smape.sum(dim=0)
            
            mse = (outputs - labels) ** 2  # Mean Square Error
            total_mse += mse.sum(dim=0)

            total_count += labels.size(0)  # number of samples

    mae = total_mae / total_count
    mape = total_mape / total_count
    smape = total_smape / total_count
    rmse = torch.sqrt(total_mse / total_count)

    return mae.cpu().numpy(), mape.cpu().numpy(), smape.cpu().numpy(), rmse.cpu().numpy()  # Moved tensors to CPU and converted to numpy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN()
model.load_state_dict(torch.load("models/newcnn_model_500images_b8_e25.pth"))
model.to(device)

mae, mape, smape, rmse = evaluate_model(model, "test")
for i in range(2):
    print(f"Metrics for prediction {i+1}:")
    print(f"Mean Absolute Error: {mae[i]:.4f}")
    print(f"Mean Absolute Percentage Error: {mape[i]:.2f}%")
    print(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%")
    print(f"Root Mean Square Error: {rmse[i]:.4f}")
    print()


Metrics for prediction 1:
Mean Absolute Error: 1.4102
Mean Absolute Percentage Error: nan%
Symmetric Mean Absolute Percentage Error: 85.29%
Root Mean Square Error: 1.6444

Metrics for prediction 2:
Mean Absolute Error: 90.2841
Mean Absolute Percentage Error: 37.10%
Symmetric Mean Absolute Percentage Error: 40.21%
Root Mean Square Error: 111.6917



In [ ]:
import torchvision.transforms as transforms
import cv2

model = CNN()
model.load_state_dict(torch.load("cnn_model.pth"))
model.eval()

# using cv2 is gross but i like the transforms more
image_path = "test\epsnx0.10_alfax-0.00_betax282.89_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = transforms.ToTensor()(image)
image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
image = image.unsqueeze(0)

with torch.no_grad():
    output = model(image)
    predicted_variables = output.squeeze().tolist()

print("Predicted variables:", predicted_variables)

Predicted variables: [0.011563804000616074, 281.4032897949219]
